---
title: Estimation from Samples
math: 
    '\abs': '\left\lvert #1 \right\rvert' 
    '\norm': '\left\lvert #1 \right\rvert' 
    '\Set': '\left\{ #1 \right\}'
    '\mc': '\mathcal{#1}'
    '\M': '\boldsymbol{#1}'
    '\R': '\mathsf{#1}'
    '\RM': '\boldsymbol{\mathsf{#1}}'
    '\op': '\operatorname{#1}'
    '\E': '\op{E}'
    '\d': '\mathrm{\mathstrut d}'
---

In [ ]:
# Execute this cell first to import the required libraries
import os

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

%matplotlib widget
if not os.getenv(
    "NBGRADER_EXECUTION"
):
    %reload_ext jupyter_ai

In [ ]:
%%ai
Add comments to the first code cell to explain the code concisely:
--
{In[1]}

This notebook will demonstrate the fundamental concepts of {term}`bias` and {term}`consistency` using Monte Carlo simulations.[^mc] 

[^mc]: See an [introduction to Monte Carlo Simulation](https://ccha23.github.io/cs1302i24a/numerical-analysis#monte-carlo-simulation).

::::{tip} How to add math to notebooks?

To include mathematical expressions in your solution, refer to the [MyST guide](https://mystmd.org/guide/math). You may organize the mathematical contents into definitions ([](#def:coin-flip)), theorems ([](#pro:confidence-interval)), and proofs ([](#prf:consistency)) using the proof directives as explained [here](https://mystmd.org/guide/proofs-and-theorems). 

::::

## Estimation Problem

Given a coin that is possibly biased, i.e., a coin toss has an unknown probability $p\in [0,1]$ of landing heads, how to estimate $p$ based on a sequence of coin tosses? More precisely:


::::{prf:definition} estimating the chance of head
:label: def:coin-flip

Consider a random process denoted by a generic random variable $\R{Z}\sim \operatorname{Bern}(p)$ that indicates whether the outcome of a coin flip is a head or not, i.e.,

$$
\R{Z} := \begin{cases}
1 & \text{if a head comes up,}\\
0 & \text{if a tail comes up.}
\end{cases}
$$ (eq:indicator)

The goal is to estimate the expectation of $\R{Z}$,

$$
E[\R{Z}] = P[\R{Z}=1] = p
$$ (eq:expectation)

given an i.i.d. sample of $\R{Z}$ of size $n$, denoted as

$$
\R{Z}^n:=(\R{Z}_1,\dots,\R{Z}_n) \text{ with }
P_{\R{Z}^n|\R{Z}} = P_{\R{Z}}^n.
$$ (eq:sample)

::::

A convenient way to obtain the i.i.d. sample is to simulate the random process using a {term}`pseudorandom number generator`:

In [ ]:
# Initialize random number generator with a seed
rng = np.random.default_rng(seed=0)

# Generate the probability of head randomly
p = rng.random()

# Set the number of coin tosses we want to simulate
n = 5000

# Use the choice function to simulate n coin tosses,
# with "H" (heads) and "T" (tails) as possible outcomes,
# and the probability of heads and tails given by p and 1-p, respectively.
coin_tosses = rng.choice(["H", "T"], size=n, p=[p, 1 - p])
coin_tosses

The above code uses **NumPy**,[^np] which is a popular python package for efficient computation on high-dimensional arrays:
- `default_rng` returns a random number generator. Setting the `seed` ensures reproducibility of the results.
- `choice` is a method of the generator to generates a random sample from a given list of possible outcomes.
- `p` keeps the value of the unknown probability $p$, which is uniformly randomly picked from the unit interval $[0,1)$.
- `coin_tosses` is a [NumPy](https://numpy.org/) array of `"H"` and `"T"`, which denote the outcomes "head" and "tail" respectively.

[^np]:  See the [official documentation](https://numpy.org/) for more details.

In [ ]:
%%ai
Why should we set a random seed in Monte-Carlo simulation? 

In [ ]:
%%ai
What is a NumPy array and why should we use it to store and process
sequences of numbers?

To obtain the i.i.d. sample $\R{Z}^n$ in [](#eq:sample), simply run

In [ ]:
coin_tosses == "H"

which gives a list of `1` (`True`) and `0` (`False`) obtained by element-wise equality comparisons with `"H"`.

## M-Estimation

A natural way to estimate $p$ is the M-estimate (sample average estimate)

$$
\begin{align}
\R{\hat{p}} &:= \frac1n \sum_{i=0}^{n-1} \R{Z}_i\\
&= \frac{\abs{\{1\leq i\leq n| \R{Z}_i=1\}} }{n},
\end{align}
$$ (eq:sample-avg)

which is the fraction of the coin tosses coming up heads. The observed distribution $(\R{\hat{p}}, 1-\R{\hat{p}})$ of heads and tails is called {term}`empirical distribution`.

The estimate can be implemented as follows using the method `mean` of a `numpy` array:

In [ ]:
(coin_tosses == "H").mean()

Is the estimate good? The following is one desirable property:

::::{prf:definition} unbiased estimate
:label: def:unbiased

An estimator $f:Z^n\to \mathbb{R}$ of $E[\R{Z}]$ from a random sample $\R{Z}^n$ is said to be *unbiased* iff

$$
E[f(\R{Z}^n)] = E[\R{Z}],
$$ (eq:unbiased)

namely, the estimate is correct in expectation.

::::

::::{exercise}
:label: ex:unbiased

The following show that the M-estimate [](#eq:sample-avg) is an unbiased estimate, i.e., [](#eq:unbiased) holds for $f(\R{Z}^n)=\R{\hat{p}}$.


:::{prf:proof} Unbiasedness
:nonumber:

$$
\begin{aligned}
E[\R{\hat{p}}] &= E\left[\frac1n \sum_{i=0}^{n-1} \R{Z}_i \right]\\
&= \frac1n \sum_{i=0}^{n-1} \underbrace{E[\R{Z}_i]}_{=p} && \text{by ???}\\
&= p.
\end{aligned}
$$ (eq:unbiased:sample-avg)

:::

What is the missing reasoning?

::::

YOUR ANSWER HERE

An unbiased estimate, while correct in expectation, can be far away from the ground truth especially when the variance is large. It is desirable for an estimate to be nearly correct with high probability, which can be stated more formally below:

::::{prf:definition} consistency

An estimator $f:Z^n\to \mathbb{R}$ of $E[\R{Z}]$ from a random sample $\R{Z}^n$ is said to be *consistent* iff

$$
\begin{align}
\lim_{n\to\infty} P[\abs{f(\R{Z}^n)- E[\R{Z}]}\leq \epsilon]=1 && \text{for all $\epsilon>0$},
\end{align}
$$ (eq:consistent)

namely, the estimate converge to the ground truth in probability.[^as-convergence]

::::

[^as-convergence]: Indeed, one can consider the stronger notion of convergence
$$
P\left[\lim_{n\to \infty} f(\R{Z}^n) = E[\R{Z}]\right] = 1,
$$
i.e., the estimate converge to the ground truth almost surely.

In [ ]:
%%ai
Explain in simple words in one paragraph the different between convergence 
in probability and almost sure convergence.

::::{exercise}
:label: ex:consistent

The M-estimate [](#eq:sample-avg) is a consistent estimate, i.e., [](#eq:consistent) holds for $f(\R{Z}^n)=\R{\hat{p}}$, due to the [law of large number (LLN)](https://en.wikipedia.org/wiki/Law_of_large_numbers).


:::{prf:proof} Consistency
:label: prf:consistency

It follows from [](#eq:unbiased:sample-avg) that

$$
\begin{aligned}
P\left[ \abs{\R{\hat{p}}-E[\R{Z}]}>\epsilon \right] &\leq \frac{\sigma^2}{\epsilon^2} && \text{by ???}
\end{aligned}
$$

for all $\epsilon>0$, where

$$
\begin{aligned}
\sigma^2 &:=\operatorname{Var}[\R{\hat{p}}]\\
&= \operatorname{Var}\left[\frac1{n} \sum_{i=0}^{n-1} \R{Z}_i\right]\\
&= \frac1{n^2} \operatorname{Var}\left[\sum_{i=0}^{n-1} \R{Z}_i\right]\\
&=\frac1{n^2} \sum_{i=1}^n \underbrace{\operatorname{Var}[\R{Z}_i]}_{=p(1-p)} && \text{by ???}\\
&=\frac{p(1-p)}{n}
\end{aligned}
$$

which goes to $0$ as desired when $n\to \infty$.

:::

What are in the missing reasonings?

::::

YOUR ANSWER HERE

In [ ]:
%%ai
Explain in one paragraph how the LLN can be used to show that 
an M-estimate is consistent.

In [ ]:
%%ai
Explain in one paragraph what Chernoff bound is and how to use it to prove the LLN.

To illustrate consistency, the following code generates and plots the estimate $\R{\hat{p}}$ for different sample size $n$.

In [ ]:
size = 5000
n = np.arange(1, size + 1)
phat = (coin_tosses == "H").cumsum() / n  # use first n tosses to estimate
sigma = (p * (1 - p) / n) ** 0.5  # true standard deviations of the estimates

# Create Figure 1, or clear it if it exists
plt.figure(1, clear=True)

# plot the ground truth p
plt.axhline(p, color="red", label=r"$p$")

# fill the region 2 sigma away from p
plt.fill_between(
    n, p - 2 * sigma, p + 2 * sigma, color="red", alpha=0.2, label=r"$p\pm 2\sigma$"
)

# plot the estimates phat
plt.plot(
    n,
    phat,
    marker=".",
    color="blue",
    linestyle="",
    markersize=1,
    label=r"$\hat{\mathsf{p}}$",
)

# configure the plot
plt.ylim([0, 1])
plt.xlim([0, n.size])
plt.title(r"Plot of ${\hat{p}}$ vs sample size")
plt.xlabel("sample size")
plt.ylabel("probability")
plt.legend()
plt.show()

For a consistent estimator, it is expected that the estimates (blue dots) converge to the ground truth (red line) as the sample size increases.

In addition, observe that the estimates mostly fall within $2$ standard deviation away from the ground truth, i.e., the convergence rate follows the rate of drop in the standard deviation:

::::{prf:proposition} confidence interval estimate
:label: pro:confidence-interval

The sample average estimates falls within $2$ standard deviation away from the ground truth over $95\%$ of the time, i.e.,

$$
P\left[\R{\hat{p}}\in [p-2\sigma, p+2\sigma]\right] \geq 0.95.
$$

The interval $[\R{\hat{p}}-2\sigma, \R{\hat{p}}+2\sigma]$ is referred to as the $95\%$-confidence interval estimate of $p$, with a confidence level of $95\%$.

::::

The proof uses the [central limit theorem (CLT)](https://en.wikipedia.org/wiki/Central_limit_theorem): As $n$ goes to $\infty$, the estimate almost surely has the gaussian/normal distribution plotted as follows:

In [ ]:
plt.figure(num=2, clear=True)

# plot the stardard normal distribution
x = np.linspace(-4, 4, 8 * 10 + 1)
plt.plot(x, norm.pdf(x), color="red", label=r"$\frac{1}{\sqrt{2\pi}}e^{-x^2/2}$")

# Fill the area under curve within certain number of standard deviations from the mean
for i in range(3, 0, -1):
    plt.fill_between(
        x,
        norm.pdf(x),
        alpha=2 ** (-i),
        color="blue",
        label=rf"$\Pr(|\hat{{p}}-p|\leq{i}\sigma)\approx {(norm.cdf(i)- norm.cdf(-i))*100:.3g}\%$",
        where=(abs(x) <= i),
    )

# Label the plot
plt.title(
    r"Standard normal distribution for $\frac{{\hat{p}}-{p}}{\sigma}$ as $n\to \infty$"
)
plt.xlabel(r"x")
plt.ylabel(r"probability density")
plt.legend()
plt.show()

In [ ]:
%%ai
Give a scenario where LLN holds but CLT does not.

In [ ]:
%%ai
Explain in simple words how to prove and understand the CLT.

:::::{seealso} What are LLN and CLT?
:class: dropdown

See the following video by [Prof. Robert Gallager](https://en.wikipedia.org/wiki/Robert_G._Gallager):

::::{card}
:header: [Open in new tab](https://www.youtube.com/embed/k0UZNZwPO8Q?si=VcQtGu935KotF8qg)
:::{iframe} https://www.youtube.com/embed/k0UZNZwPO8Q?si=VcQtGu935KotF8qg
:width: 100%
:::
::::

:::::

## A Coin Tossing Game

To understand the concept of bias in estimation, imagine playing the coin-tossing game:

- You win if a coin toss comes up head.
- You get to choose 1 out of the $m$ coins $i\in \{0,\dots,m-1\}$ with unknown probability $p_i$ of coming up head.
- You can flip each coin $n$ times before making your choice.

**How to play the game?**

A particular strategy for playing the game is to 
1. estimate the chance $p_i$ by the empirical probability $\R{\hat{p}}_i$ for each coin $i$, and
1. select the coin (with ties broken arbitrarily)

  $$
  \R{J} := \arg\max_i \R{\hat{p}}_i.
  $$

It is easy to see that the chance of winning by the given strategy is $E[p_{\R{J}}]$. Is the strategy optimal? Can a player evaluate or estimate the chance of winning without knowing $p_i$'s? Is the following a good estimate:

$$
\R{\hat{p}}_{\R{J}} = \max_i\R{\hat{p}}_i
$$

::::{important} How is the strategy related to data-mining?

Suppose $\R{\hat{p}}_i$ is the empirical accuracy of the classifier $f_i$. A common model selection strategy is to 
- choose the classifier $\R{J}$ defined above that has the highest empirical accuracy, and
- estimate its performance by $\R{\hat{p}}_{\R{J}}$.

::::

**How to evaluate the estimate?**

Consider the simple case $n=1$, $m=2$, and $p_0=p_1=0.5$. We have the following four equally likely events:

$$
\begin{array}{ccc} \R{\hat{p}}_0 & \R{\hat{p}}_1 & \max_i \R{\hat{p}}_i \\\hline
0 & 0 & 0\\ 
0 & 1 & 1\\ 
1 & 0 & 1\\ 
1 & 1 & 1\\ 
\end{array}
$$

::::{exercise}
:label: ex:n=1

For the above simple case, compute $E[p_{\R{J}}]$ and $E[\max_i\R{\hat{p}}_i]$. Is $\max_i\R{\hat{p}}_i$ an unbiased estimate of $E[p_{\R{J}}]$?

::::

YOUR ANSWER HERE

Instead of carrying out the exact analysis, which involves [order statistics][os], we will conduct the [Monte-Carlo simulation][MC] of the coin tossing game. The simulation can be verified by hand-calculating the closed-form solution for $n=1$, $m=2$, and $p_0=p_1=0.5$.

[MC]: https://www.cs.cityu.edu.hk/~ccha23/cs1302book/Lecture9/Monte%20Carlo%20Simulation%20and%20Linear%20Algebra.html
[os]: https://en.wikipedia.org/wiki/Order_statistic

The following initializes the list `p_list` of probabilities of head for different coins:

In [ ]:
m = 2
p_list = np.array([0.4] * (m - 1) + [0.6])
# To generate the probability randomly instead, use
# p_list = rng.random(m)
p_list

Instead of generating a sequence of coin tosses, we will simulate $\R{\hat{p}}_i$ directly using the binomial distribution since

$$
n\R{\hat{p}}_i\sim \operatorname{Binomial}(n,p_i).
$$

In [ ]:
size = 10
n = np.arange(1, size + 1)
k = 100000
phat = rng.binomial(
    n.reshape(-1, 1, 1), p_list.reshape(1, -1, 1), (size, m, k)
) / n.reshape(-1, 1, 1)
max_phat = phat.max(axis=1)
max_phat

`max_phat` is a 2-dimensional array of samples of $\max_{i}\R{\hat{p}}_i$:
- The first axis indexes samples obtained from different number of tosses.
- The second axis indexes `k` independent samples for the same number of tosses.

The `k` independent samples can be used to approximates $E[\max_{i}\R{\hat{p}}_i]$ as follows.

In [ ]:
E_max_phat = max_phat.mean(axis=-1)
E_max_phat

Similarly, the winning probability can be approximated as follows:

In [ ]:
win_prob = p_list[phat.argmax(axis=1)].mean(axis=-1)
win_prob

The following plots compare the probabilities as a function of $n$.

In [ ]:
plt.figure(3, clear=True)
plt.axhline(p_list.max(), color="red", label=r"$\max_i p_i$")
plt.plot(
    n,
    E_max_phat,
    linestyle="--",
    marker=".",
    color="blue",
    markersize=10,
    label=r"$E[\max_i{\hat{p}}_i]$",
)
plt.plot(
    n,
    win_prob,
    linestyle=":",
    marker="x",
    color="green",
    markersize=10,
    label="winning probability",
)

plt.ylim([0, 1])
plt.xlim([n[0], n[-1]])
plt.title(r"Plot of $E[\max_i{\hat{p}}_i]$ vs $n$")
plt.xlabel("$n$")
plt.ylabel("probability")
plt.legend()
plt.show()

::::{exercise}
:label: ex:max_p

Compare the chance of winning with $\max_i p_i$ more generally for different $p_i$'s.

:::{hint}
:class: dropdown

Change `p_list` to explore the non-uniform cases where $p_i$'s may not be equal. Be careful about the deterministic case where $p_i\in \Set{0,1}$ for all $i$.

:::
::::

YOUR ANSWER HERE

::::{exercise}
:label: ex:max_phat:bias
 Compare the chance of winning with $E[\max_i \R{\hat{p}}_i]$. Is $\max_i \R{\hat{p}}_i$ a biased estimate? If so, is it overly optimistic, i.e., has an expectation larger than the chance of winning?

::::

YOUR ANSWER HERE

::::{exercise} 
:label: ex:max_phat:consistency

Is $\max_i \R{\hat{p}}_i$ a consistent estimate of the chance of winning?
::::

YOUR ANSWER HERE

In [ ]:
%%ai
State in simple words the uniform LLM and how it differs from the usual LLM.
Explain what makes uniform convergence possible and why it can be applied to
data mining and machine learning?

## Glossary

:::::{admonition}
:class: dropdown

::::{glossary}

bias
: The discrepancy between the expected estimate from an estimator and the ground truth of the unknown being estimated.

consistency
: The property of an estimator whereby a sequence of estimates converges in probability to the ground truth as the sample size increases.

empirical distribution
: The frequencies of the different values observed in a dataset.

pseudorandom number generator
: A algorithm to produce a seemingly random but actually deterministic sequence of numbers.

::::

:::::